In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from src.data_prep.Nifty50Dataset import Nifty50Dataset
from src.forecasting.GaussianTransformer import CustomTransformer
from src.forecasting.StockSeriesForcaster import StockSeriesForecaster

model = CustomTransformer(d_model=4, nhead=2)
dataset = Nifty50Dataset("/Users/maniksomayaji/Documents/omscs/capstone_project/data/NIFTY 50_minute_data.csv", window_size=300, prediction_horizon=1)
loader = DataLoader(dataset, batch_size=2, shuffle=False)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [25]:
forecaster = StockSeriesForecaster(model, optimizer, criterion)
forecaster.train(loader)

[Epoch 1 | Batch 50] Loss: 1.2460
[Epoch 1 | Batch 100] Loss: 0.5858
[Epoch 1 | Batch 150] Loss: 1.1691
[Epoch 1 | Batch 200] Loss: 1.3598
[Epoch 1 | Batch 250] Loss: 0.5049
[Epoch 1 | Batch 300] Loss: 0.6500
[Epoch 1 | Batch 350] Loss: 0.8728
[Epoch 1 | Batch 400] Loss: 2.2706
[Epoch 1 | Batch 450] Loss: 1.5186
[Epoch 1 | Batch 500] Loss: 1.6571
[Epoch 1 | Batch 550] Loss: 0.6299
[Epoch 1 | Batch 600] Loss: 0.5198
[Epoch 1 | Batch 650] Loss: 1.9753
[Epoch 1 | Batch 700] Loss: 1.4216
[Epoch 1 | Batch 750] Loss: 1.1752
[Epoch 1 | Batch 800] Loss: 1.1328
[Epoch 1 | Batch 850] Loss: 0.1866
[Epoch 1 | Batch 900] Loss: 1.2965
[Epoch 1 | Batch 950] Loss: 0.7006
[Epoch 1 | Batch 1000] Loss: 1.1182
[Epoch 1 | Batch 1050] Loss: 0.8796
[Epoch 1 | Batch 1100] Loss: 0.9895
[Epoch 1 | Batch 1150] Loss: 0.3816
[Epoch 1 | Batch 1200] Loss: 0.4443
[Epoch 1 | Batch 1250] Loss: 0.4421
[Epoch 1 | Batch 1300] Loss: 0.6290
[Epoch 1 | Batch 1350] Loss: 0.8260
[Epoch 1 | Batch 1400] Loss: 0.3489
[Epoch 1 | B

In [29]:
num_batches = 3
for i, (x, y) in enumerate(loader):
    if i >= num_batches:
        break
    output = forecaster.autoregressive_predict(x, max_len=20)
    print("Generated:", output)

Generated: tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.2743,  0.0860, -0.3707, -0.5824],
         [-0.2746,  0.0860, -0.3706, -0.5820],
         [-0.2748,  0.0860, -0.3706, -0.5816]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.3171,  0.0863, -0.3387, -0.5221],
         [-0.3173,  0.0863, -0.3386, -0.5218],
         [-0.3175,  0.0863, -0.3384, -0.5215]]])
Generated: tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.2890,  0.0861, -0.3631, -0.5589],
         [-0.2893,  0.0861, -0.3630, -0.5585],
         [-0.2896,  0.0861, -0.3628, -0.5581]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000, 

In [ ]:
import matplotlib.pyplot as plt
import torch

data_x, data_y = dataset[0]
full_series = data_x[:320]

input_seq = full_series[:300].unsqueeze(0)
true_future = full_series[300:]

with torch.no_grad():
    predicted_future = forecaster.autoregressive_predict(input_seq, max_len=20)
predicted_future = predicted_future.squeeze()  # Remove batch dimension

plt.figure(figsize=(12, 6))
plt.plot(range(300), input_seq.squeeze(), label='Input (past 300)')
plt.plot(range(300, 320), true_future, label='Actual Future (next 20)', color='green')
plt.plot(range(300, 320), predicted_future, label='Predicted Future (next 20)', color='red', linestyle='--')
plt.legend()
plt.title('Actual vs. Predicted Forecast')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.grid(True)
plt.tight_layout()
plt.show()